# Machine Learning - Assignment 1

## Naive Bayes learning algorithm for binary classification, Cross-validation, and ROC-Curves

The aim of the assignment is to implement:

* Naive Bayes learning algorithm for binary classification tasks
* Visualization to plot a ROC-curve
* A cross-validation test
* Visualization of the average ROC-curve of a cross-validation test

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries and reading the data.

Look at the output to get an idea of how the data is structured.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import e, pi, sqrt

data = pd.read_csv("iris.csv").to_numpy()

mapped, index, unique_arr = np.unique(data[:, -1], return_index=True, return_inverse=True)
data[:, -1] = unique_arr
iris_setosa, iris_versicolor, iris_virginica = np.split(data, index[1:])

print(f"Full data array (features and labels):\n{iris_setosa[:3]}\n")
print("###############\n")
print(f"Train features (first 4 columns):\n{iris_setosa[:3, :-1]}\n")
print(f"Labels (last column):\n{iris_setosa[:3, -1:]}\n")
print(f"Names of labels:\n{[[numb, name] for numb, name in enumerate(mapped)]}")

Full data array (features and labels):
[[5.1 3.5 1.4 0.2 0]
 [4.9 3.0 1.4 0.2 0]
 [4.7 3.2 1.3 0.2 0]]

###############

Train features (first 4 columns):
[[5.1 3.5 1.4 0.2]
 [4.9 3.0 1.4 0.2]
 [4.7 3.2 1.3 0.2]]

Labels (last column):
[[0]
 [0]
 [0]]

Names of labels:
[[0, 'Iris-setosa'], [1, 'Iris-versicolor'], [2, 'Iris-virginica']]


In [2]:
# Example print of the 3 first datapoints (similar as above):
iris_setosa[:3]

array([[5.1, 3.5, 1.4, 0.2, 0],
       [4.9, 3.0, 1.4, 0.2, 0],
       [4.7, 3.2, 1.3, 0.2, 0]], dtype=object)

## Data handling functions

As a start, we are going to implement some basic data handling functions to use in the future.

### 1) Split class into a train and test set

First, we need to be able to split the class into a train and test set.

For this assignment, we recommend a 80/20-split, that is 80% of the data is used for training, 20% is used for testing.

In [3]:
def train_test_split(class_data, train_ratio=0.8):
    
    # TODO: Split the data into two parts, train and test.
    # Numpy has some functionallity that makes implementing this part easier.
    np.random.shuffle(class_data)
    split_index = int(len(class_data) * train_ratio)
    train = class_data[:split_index]
    test = class_data[split_index:]

    # NOTE: Try to use the default parameter "train_ratio" which is currently set to a default value of 0.8.

    return train, test

In [4]:
# TODO: Test the train_test_split function
train, test = train_test_split(data)

# TODO: Print the output
print(train)
print(test)

[[6.0 3.0 4.8 1.8 2]
 [5.0 3.4 1.6 0.4 0]
 [5.1 2.5 3.0 1.1 1]
 [4.6 3.4 1.4 0.3 0]
 [6.7 3.1 4.7 1.5 1]
 [5.5 2.4 3.7 1.0 1]
 [6.1 2.8 4.0 1.3 1]
 [6.9 3.2 5.7 2.3 2]
 [5.2 3.5 1.5 0.2 0]
 [7.0 3.2 4.7 1.4 1]
 [4.7 3.2 1.3 0.2 0]
 [6.6 3.0 4.4 1.4 1]
 [5.2 3.4 1.4 0.2 0]
 [6.8 3.2 5.9 2.3 2]
 [5.9 3.2 4.8 1.8 1]
 [6.3 2.9 5.6 1.8 2]
 [5.1 3.3 1.7 0.5 0]
 [4.6 3.1 1.5 0.2 0]
 [7.7 3.8 6.7 2.2 2]
 [5.4 3.4 1.5 0.4 0]
 [7.4 2.8 6.1 1.9 2]
 [7.2 3.6 6.1 2.5 2]
 [6.1 3.0 4.9 1.8 2]
 [5.0 2.0 3.5 1.0 1]
 [5.4 3.7 1.5 0.2 0]
 [6.2 3.4 5.4 2.3 2]
 [5.7 2.6 3.5 1.0 1]
 [7.9 3.8 6.4 2.0 2]
 [5.0 3.5 1.6 0.6 0]
 [6.2 2.9 4.3 1.3 1]
 [5.8 2.7 3.9 1.2 1]
 [6.1 2.6 5.6 1.4 2]
 [6.4 3.2 4.5 1.5 1]
 [6.5 3.0 5.2 2.0 2]
 [5.0 2.3 3.3 1.0 1]
 [5.1 3.7 1.5 0.4 0]
 [4.7 3.2 1.6 0.2 0]
 [4.9 2.5 4.5 1.7 2]
 [5.1 3.5 1.4 0.2 0]
 [6.9 3.1 5.4 2.1 2]
 [5.6 3.0 4.5 1.5 1]
 [6.4 3.2 5.3 2.3 2]
 [4.8 3.0 1.4 0.1 0]
 [5.1 3.8 1.5 0.3 0]
 [6.2 2.8 4.8 1.8 2]
 [5.8 2.7 4.1 1.0 1]
 [6.7 3.3 5.7 2.5 2]
 [5.5 2.4 3.8

### 2) Split data into features and labels

The data as shown above is not always the optimal shape. To help us keep track of things, we can split the data into its features and labels seperately.

Each class is 4 features and 1 label in the same array: 

* **[feature 1, feature 2, feature 3, feature 4, label]**

It would help us later to have the features and labels in seperate arrays in the form: 

* **[feature 1, feature 2, feature 3, feature 4]** and **[label]**

Here you are going to implement this functionallity.

In [6]:
def feature_label_split(data):

    # TODO: Split one class into its features and labels
    features = data[:, :-1]
    labels = data[:, -1]
    
    return features, labels

We should first test the "**Feature_label_split**" function on one of the classes above (iris_setosa, etc...) to make sure it works properly.

In [14]:
# TODO: Test the class splitting function
train_setosa, test_setosa = train_test_split(iris_setosa)

# TODO: Print the output
print(train_setosa)
print(test_setosa)

[[6.0 3.0 4.8 1.8 2]
 [4.3 3.0 1.1 0.1 0]
 [5.1 3.8 1.5 0.3 0]
 [5.4 3.4 1.5 0.4 0]
 [5.2 3.5 1.5 0.2 0]
 [4.6 3.4 1.4 0.3 0]
 [5.5 2.4 3.8 1.1 1]
 [7.7 3.8 6.7 2.2 2]
 [6.2 2.9 4.3 1.3 1]
 [5.4 3.7 1.5 0.2 0]
 [4.6 3.1 1.5 0.2 0]
 [6.9 3.1 5.4 2.1 2]
 [4.7 3.2 1.6 0.2 0]
 [6.3 2.9 5.6 1.8 2]
 [7.0 3.2 4.7 1.4 1]
 [5.1 3.7 1.5 0.4 0]
 [4.9 2.5 4.5 1.7 2]
 [5.2 3.4 1.4 0.2 0]
 [6.4 3.2 5.3 2.3 2]
 [5.1 2.5 3.0 1.1 1]
 [6.8 3.2 5.9 2.3 2]
 [6.4 3.2 4.5 1.5 1]
 [5.6 3.0 4.5 1.5 1]
 [5.0 3.5 1.6 0.6 0]
 [5.0 2.0 3.5 1.0 1]
 [5.8 2.7 3.9 1.2 1]
 [5.8 2.7 4.1 1.0 1]
 [7.9 3.8 6.4 2.0 2]
 [5.0 3.4 1.6 0.4 0]
 [5.7 2.6 3.5 1.0 1]
 [6.2 2.8 4.8 1.8 2]
 [5.7 3.8 1.7 0.3 0]
 [4.7 3.2 1.3 0.2 0]
 [6.6 3.0 4.4 1.4 1]
 [6.1 3.0 4.9 1.8 2]
 [4.8 3.0 1.4 0.1 0]
 [6.7 3.3 5.7 2.5 2]
 [6.2 3.4 5.4 2.3 2]
 [6.1 2.8 4.0 1.3 1]
 [6.5 3.0 5.2 2.0 2]]
[[6.1 2.6 5.6 1.4 2]
 [6.7 3.1 4.7 1.5 1]
 [6.9 3.2 5.7 2.3 2]
 [7.4 2.8 6.1 1.9 2]
 [5.1 3.5 1.4 0.2 0]
 [5.0 2.3 3.3 1.0 1]
 [5.9 3.2 4.8 1.8 1]
 [5.1 3.3 1.

We should also try to **1)** first split a class into a train and test set, **2)** split each of these two into features and abels. In total there should be 4 arrays (2 feature and 2 label arrays).

In [15]:
# TODO: Use the above data handling functions to make 2 feature arrays and 2 label arrays. 
features_train_setosa, labels_train_setosa = feature_label_split(train_setosa)
features_test_setosa, labels_test_setosa = feature_label_split(test_setosa)


# TODO: Print the output (you may use multiple print-statements if you wish)
print(features_train_setosa)
print(labels_train_setosa)
print(features_test_setosa)
print(labels_test_setosa)

[[6.0 3.0 4.8 1.8]
 [4.3 3.0 1.1 0.1]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.5 0.4]
 [5.2 3.5 1.5 0.2]
 [4.6 3.4 1.4 0.3]
 [5.5 2.4 3.8 1.1]
 [7.7 3.8 6.7 2.2]
 [6.2 2.9 4.3 1.3]
 [5.4 3.7 1.5 0.2]
 [4.6 3.1 1.5 0.2]
 [6.9 3.1 5.4 2.1]
 [4.7 3.2 1.6 0.2]
 [6.3 2.9 5.6 1.8]
 [7.0 3.2 4.7 1.4]
 [5.1 3.7 1.5 0.4]
 [4.9 2.5 4.5 1.7]
 [5.2 3.4 1.4 0.2]
 [6.4 3.2 5.3 2.3]
 [5.1 2.5 3.0 1.1]
 [6.8 3.2 5.9 2.3]
 [6.4 3.2 4.5 1.5]
 [5.6 3.0 4.5 1.5]
 [5.0 3.5 1.6 0.6]
 [5.0 2.0 3.5 1.0]
 [5.8 2.7 3.9 1.2]
 [5.8 2.7 4.1 1.0]
 [7.9 3.8 6.4 2.0]
 [5.0 3.4 1.6 0.4]
 [5.7 2.6 3.5 1.0]
 [6.2 2.8 4.8 1.8]
 [5.7 3.8 1.7 0.3]
 [4.7 3.2 1.3 0.2]
 [6.6 3.0 4.4 1.4]
 [6.1 3.0 4.9 1.8]
 [4.8 3.0 1.4 0.1]
 [6.7 3.3 5.7 2.5]
 [6.2 3.4 5.4 2.3]
 [6.1 2.8 4.0 1.3]
 [6.5 3.0 5.2 2.0]]
[2 0 0 0 0 0 1 2 1 0 0 2 0 2 1 0 2 0 2 1 2 1 1 0 1 1 1 2 0 1 2 0 0 1 2 0 2
 2 1 2]
[[6.1 2.6 5.6 1.4]
 [6.7 3.1 4.7 1.5]
 [6.9 3.2 5.7 2.3]
 [7.4 2.8 6.1 1.9]
 [5.1 3.5 1.4 0.2]
 [5.0 2.3 3.3 1.0]
 [5.9 3.2 4.8 1.8]
 [5.1 3.3 1.7 0.5]
 [7.

Think a bit before going to the next task, what can easily go wrong in the above code?

## Naive Bayes learning algorithm

When implementing the Navie Bayes learning algorithm, we can break it down into its components similar to how Naive Bayes is formulated.

Common formulation:
$$
P(Y \mid X_i) = \frac{P(Y)\,P(X_i \mid Y)}{P(X_i)}
$$

In 'Plain English':
$$
\text{Posterior} = \frac{\text{Prior} \times \text{Likelihood}}{\text{Evidence}}
$$

In short, we are looking for the **posterior** based on the **prior**, **likelihood** and **evidence**.

In practice, we will calculate the **posterior** for each class and then we look for the class with the highest (max) **posterior**.

A great source of learning is the Wiki page and the example towards the end:
https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Person_classification

Below are the components which are needed to calculate the posterior for each class.

### 3) Calculate feature statistics

First, we need to implement a function that returns feature statistics (means, standard deviation, and priors) for a single set of feature data for a single class. This is the equivalent of "training" the naive bayes model.

**Note 1:** Each feature for each class gets its own mean, standard deviation, and priors!

**Note 2:** The way you structure the functions (what is returned) shapes the remainder of the assignment.

In [51]:
def calculate_feature_statistics(feature_data):

    # TODO: Complete this function so that it returns the feature statistics for the input data.
    means = []
    std_devs = []
    priors = 0
    for i in range(feature_data.shape[1]):
        means.append(np.mean(feature_data[:, i]))
        std_devs.append(np.std(feature_data[:, i]))
    priors = len(feature_data) / len(data)

    return np.array(means), np.array(std_devs), priors

To make sure the function works, we should test it before proceding.

In [52]:
# TODO: Make sure to use our previous class splitting function.
features_setosa, labels_setosa = feature_label_split(iris_setosa)

# TODO: Test the function here for one of the dataset classes. 
means, std_devs, priors = calculate_feature_statistics(features_setosa)

# TODO: Print the output from the feature statistic function.
print("Means:", means)
print("Standard Deviations:", std_devs)
print("Priors:", priors)

Means: [5.822 3.102 3.68  1.192]
Standard Deviations: [0.8665541  0.41975707 1.75077126 0.7628473 ]
Priors: 0.3333333333333333


### 4) Gaussian probability density function (Gaussian PDF)

Now we need to implement the gaussian probability density function to use for a single datapoint.

**Note:** Look at the imports in the first cell at the top, it has some math numbers for easy use here. 

In [64]:
def gaussian_probability_density_function(x, means, stdevs):

    if np.isscalar(x):  # Handle scalar input (for testing)
        probability_density = (1 / (stdevs * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - means) / stdevs) ** 2)
    else:  # Handle array input (for prediction)
        probability_density = []
        for i in range(len(x)):
            p = (1 / (stdevs[i] * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x[i] - means[i]) / stdevs[i]) ** 2)
            probability_density.append(p)
        probability_density = np.array(probability_density)
    return probability_density

### 5) Testing Gaussian PDF

We should test it to make sure it works. Train it, using the "calculate_feature_statistics" function, on one of the dataset classes. Then, take one datapoint from the same class and use naive bayes gaussian to calculate one posterior for one class.

In [65]:
# TODO: Implement the code below to test the "gaussian_probability_density_function" function for one of the classes.

means, std_devs, priors = calculate_feature_statistics(features_setosa)

# TODO: Test with one datapoint from the learned class.
probability_density = gaussian_probability_density_function(iris_setosa[0, 0], means[0], std_devs[0])

# TODO: Print the probability density
print("Probability Density:", probability_density)

Probability Density: 0.26139067682561795


As a test, take one datapoint from one of the other classes and see if the predicted probability changes.

In [66]:
# TODO: Take one datapoint from one of the other classes.
other_class_datapoint = iris_versicolor[0, 0]
means_oc, std_devs_oc, priors_oc = calculate_feature_statistics(iris_versicolor)

# TODO: Use naive bayes gaussian on this datapoint with the same feature statistics as the first class.
other_class_probability = gaussian_probability_density_function(other_class_datapoint, means_oc[0], std_devs_oc[0])

# TODO: Print the probability
print("Other Class Probability Density:", other_class_probability)

Other Class Probability Density: 0.42523762894599537


Think a bit why the probability changes, what could affect the prediction?

## Prepare Naive Bayes for binary classification

### 6) Prepare the data for inference

Before we train and test the naive bayes for multiple classes, we should get our data in order.

Similar to how we did previously, we should now split two classes into a train and test set, you may choose which two classes freely.

In [67]:
# TODO: Split two classes into train and test sets.
train_setosa, test_setosa = train_test_split(iris_setosa)
train_versicolor, test_versicolor = train_test_split(iris_versicolor)


# TODO: Sepearte the features and lables for both the train and test set.
features_train_setosa, labels_train_setosa = feature_label_split(train_setosa)
features_test_setosa, labels_test_setosa = feature_label_split(test_setosa)
features_train_versicolor, labels_train_versicolor = feature_label_split(train_versicolor)
features_test_versicolor, labels_test_versicolor = feature_label_split(test_versicolor)

### 7) Class A vs Class B for binary classification

We have to get the posterior from two classes and compare the two posteriors in order to make a propper prediction.

For this, we need a function that calculates the posteriors for each class in binary classification and outputs the one with the highest posterior.

**IMPORTANT NOTES**:
1) When we are calculating the **Likelihood**, we are multiplying a small for each feature, making the result very very small. This can be problematic if we have a lot of features. Because of this, to make the calculation numerically stable, we instead calculate the sum of log feature likelihood:
$$
\prod_{i} P(X_i \mid Y) \rightarrow \sum_{i} \text{log}( P(X_i \mid Y) )
$$
2) We are **not** calculating probabilities, we are calculating **priors** and **likelihoods**. A lot of people make this misstaken. **Priors** and **likelihoods** can be less or more than 1 (even when summed up) and this is normal behavior.


In [68]:
def naive_bayes_prediction(data_point, feature_stats):

    # TODO: Implement the rest of this function. Make use of previous functions that you have implemented.
    posteriors = {}
    for class_label, stats in feature_stats.items():
        means, std_devs, prior = stats
        likelihood = sum(np.log(gaussian_probability_density_function(data_point, means, std_devs)))
        posterior_score = likelihood + np.log(prior)
        posteriors[class_label] = posterior_score


    # NOTE: We need to compare the posteriors at some point between the classes we test on, the predicted class should be the class with the highest posterior.

    prediction_class = max(posteriors, key=posteriors.get)
    return prediction_class

To test the function we need the feature statistics from the classes we choose. 

**Note:** Choose the correct train/test set and the correct feature/label split!

In [69]:
# TODO: Get the feature metrics for the classes.
feature_stats = {
    0: calculate_feature_statistics(features_train_setosa),
    1: calculate_feature_statistics(features_train_versicolor)
}

Now we should have implemented all the neccessary parts to train a naive bayes algorithm and do inference on it. Implement a small test workflow for two of your chosen classes.

In [ ]:
# TODO: Test the "naive_bayes_prediction" function and implement all neccessary code for it to work.
# Combine test features and labels
test_features = np.vstack((features_test_setosa, features_test_versicolor))
test_labels = np.hstack((labels_test_setosa, labels_test_versicolor))


# TODO: Print the predicted class and the actual class for the test set
for i, feature in enumerate(test_features):
    prediction = naive_bayes_prediction(feature, feature_stats)
    print(f"Predicted class: {prediction}, Actual class: {test_labels[i]}")
# Accuracy
accuracy = np.sum([naive_bayes_prediction(feature, feature_stats) == test_labels[i] for i, feature in enumerate(test_features)]) / len(test_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Predicted class: 0, Actual class: 0
Predicted class: 0, Actual class: 0
Predicted class: 0, Actual class: 0
Predicted class: 1, Actual class: 1
Predicted class: 1, Actual class: 1
Predicted class: 1, Actual class: 2
Predicted class: 1, Actual class: 2
Predicted class: 1, Actual class: 2
Predicted class: 1, Actual class: 1
Predicted class: 0, Actual class: 2
Predicted class: 0, Actual class: 0
Predicted class: 0, Actual class: 1
Predicted class: 0, Actual class: 0
Predicted class: 1, Actual class: 2
Predicted class: 1, Actual class: 0
Predicted class: 1, Actual class: 1
Predicted class: 1, Actual class: 1
Predicted class: 1, Actual class: 1
Predicted class: 0, Actual class: 1
Predicted class: 0, Actual class: 1
Accuracy: 55.00%


## ROC-curve

A ROC curve, or *Receiver Operating Characteristic curve*, is a graphical plot that illustrates the performance of a binary classifier such as our Naive Bayes model.

More info can be found in the course material and here: [https://en.wikipedia.org/wiki/Receiver_operating_characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

Another good illustration by Google can be found here: [https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

Now that we have a prediction model, we would want to try it out and test it using a ROC-curve.

### 8) True Positive Rate (TPR) and False Positive Rate (FPR)

From our prediction function we get probabilities, and for prediction purposes we have just predicted the one with the highest probability.

To plot a ROC-curve, we need the TPR and FPR for the binary classification. We will implement this here.

**Note 1:** The threshold is is a value that goes from 0 to 1. 

**Note 2:** One of the two classes will be seen as "the positive class" (prediction over the threshold) and the other as "the negative class" (prediction under the threshold).

**Note 3:** The threshold stepsize will decide the size of the returned TPR/FPR list. A value of 0.1 will give 10 elements (0 to 1 in increments of 0.1)

In [ ]:
# Stepsize demonstration
print("Python list:", [x/10 for x in range(0,10,1)])

# Stepsize demonstration with numpy:
print("Numpy linspace:", np.linspace(0,1,11))
print("Numpy linspace (no endpoint):", np.linspace(0,1,10,endpoint=False))

Python list: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
Numpy linspace: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Numpy linspace (no endpoint): [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [ ]:
def TPR_and_FPR(prediction_probabilities, test_labels, threshold_stepsize = 0.1):

    # TODO: Implement the rest of this function

    TPR = ...
    FPR = ...

    return TPR, FPR

In [ ]:
# TODO: Test the "TPR_and_FPR" function on the model you have created previously.

How does the values change if you change the threshold stepsize? 

How does the values change if you change the classes you compare?

### 9) Plot the TPR and FPR

To better see what is going on, we can plot the TPR and FPR. We can also calculate the Area Under the ROC Curve (AUC or AUROC) at the same time.

In [ ]:
def plot_ROC(TPR, FPR):

    # TODO: Calculate the AUC score.

    # TODO: Plot the TPR and FPR using plt (matplotlib)

    # TODO: Add a "middle-line" in the plot. This can be seen as the "better/worse than random" line.

    plt.plot(...)

    plt.show()

    pass # No need to return anything, remove this line once you are finished implementing.

In [ ]:
# TODO: Test the plotting function on the TPR and FPR you just calculated.

## Cross-validation

The final task is to take everything you have implemented so far and apply it in a cross-validation loop.

**Note 1:** To better reflect a real scenarios, you should shuffle the data before doing cross-validation.

**Note 2:** When using cross-validation, the interesting thing is the mean performance (mean AUC, mean accuracy, mean ROC-curve).

**Note 3:** This part is a bit more free in terms of implementation, but make sure to use some of the previously implemented functions.

### 10) Cross-validation loop

In [ ]:
def cross_validation(data, target, folds = 10, threshold_stepsize = 0.1):

    # TODO: Implement Cross-validation
    
    for i in range(folds):


        ...

    # TODO: Plot the average ROC-curve.
    # NOTE: Take the correct average!

    plot_ROC(...)

### 11) 10-fold Cross-validation on all classes

Test the "cross_validation" function on all the classes against eachother using 10 folds.

* Iris-setosa vs Iris-versicolor
* Iris-setosa vs Iris-virginica
* Iris-versicolor vs Iris-virginica

In [ ]:
# TODO: Implement and test cross-validation function on all classes.

In [ ]:
# TODO: Implement and test cross-validation function on all classes.

In [ ]:
# TODO: Implement and test cross-validation function on all classes.

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is it called "naive bayes"?

2) What are some downsides of the naive bayes learning algorithm?

3) When using ROC-curves, what is the theoretical best and worst result you can get?

4) When using ROC-curves, in this assignment for example, is a higher threshold-stepsize always better?  


# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!